# Mutation Types and Genomic Element Types in `shadie`

## Mutation Types
Mutation types must be defined first in `shadie` using the `MutationType` class. `MutationType` requires a minimum of 3 arguments:
* dominance (float): dominance coefficient of the mutation for diploid
* distribution (str): distribution that the fitness effect will be selected from **to add: how to access a list and explanation of distributions**
* params (float): additional arguments that define the distribution - number of arguments will vary based on the distribution 

`shadie` defaults:
```python
NEUT = MutationType(0.5, "f", 0.0)		#neutral mutation
SYN = MutationType(0.5, "f", 0.0)         #synonymous
DEL = MutationType(0.1, "g", -0.03, 0.2)  #deleterious
BEN = MutationType(0.8, "e", 0.1)         #beneficial
```

In [2]:
from shadie import MutationType

In [3]:
#"f" is fixed fitness effect (no distribution), so takes a single argument
mut1 = MutationType(0.5, "f", 0.1)

#"g" is a gamma distribution and takes 2 arguments: alpha, beta
mut2 = MutationType(0.1, "g", -0.03, 0.2)
mut3 = MutationType(0.5, "n", 0.05, 0.1)
mut4 = MutationType(0.5, "w", 5.5, 2.4)

01:42 | WARNING | __init__        | 'g', 'n', and 'w' distributions take 2 params
01:42 | WARNING | __init__        | 'e' and 'f' distributions take 1 param
01:42 | WARNING | __init__        | 'e' and 'f' distributions take 1 param
01:42 | WARNING | __init__        | 'e' and 'f' distributions take 1 param


You must assign each created mutation type to an object in order to save them to a `MutationList`, which is needed for your simulation. The object name you assign will be saved as a "name" ___ in the `MutationList` and you will be able to refer to each mutation by this name *or* the `shadie`-assigned `idx` when creating your genomic element types. 

The repr lists the mutation settings:

In [4]:
mut4

<MutationType: m8, 0.5,w, (5.5, 2.4)>

Notice that your mutation type now has an `idx` (in the format m#). This is a unique id that `shadie` uses to keep track of mutations. It may not start at `m1`, as some default mutation types have already been defined by `shadie`. 

If you'd like more information about your mutation, you can use the `inspect()` function to visualize your mutation distribution and see the parameters listed explicitly:

In [5]:
mut2.inspect()

Mutation Type
idx: m6
dominance coefficient: 0.1
distribution: g
distribution parameters: (-0.03, 0.2)
Distribution plot:


mean: -0.03
standard deviation: 0.0670820393249937





## Save your custom mutations to a list
Create a `MutationList` class object that contains all the mutations you would like to use in your simulation. You will need to call this list when you initiate the script. If you are using custom mutations in adding to defaults, you will have to include those defaults in your new list.

In [9]:
from shadie import MutationList

In [10]:
#list containing custom mutations
mylist = MutationList(mut1,  mut2, mut3, mut4)
mylist

<MutationList: ['m5', 'm6', 'm7', 'm8']>

In [11]:
mymixedlist = MutationList(mut1,  mut2, mut3, mut4, BEN, NEUT, DEL)

NameError: name 'BEN' is not defined

## Genomic Element Types
Each genomic element type describes a region of chromosome. It is helpful to think of these regions as "non=coding", "exon", "intron", etc. 
Each type requires 2 lists of equal length:
1. A list of mutations that can occur in that kind of region (e.g. only neutral mutations can occur in non-coding regions)
2. A list of relative frequencies 

`shadie` defaults:
```python
EXON = ElementType([SYN, DEL, BEN], (2,8,0.1))  #exon
INTRON = ElementType([SYN,DEL], (9,1))          #intron
NONCOD = ElementType(NEUT, 1)              	 #non-coding
```